# Facial Recognition

In [ ]:
%pip install torch
%pip install facenet_pytorch

In [38]:
import torch
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

## Reading input data

In [ ]:
# Using FaceNet to obtain embeddings (vector) of faces
# Reading npy files
embeddings = np.load('../data/embeddings.npy')
labels = np.load('../data/labels.npy')

# Getting LabelEncoder
with open('../data/label_encoder.pkl', 'rb') as f:
  label_encoder = pickle.load(f)

# Debug
print(embeddings.shape)
print(labels.shape)
print(label_encoder.inverse_transform(labels)[0])

## Training and evaluating the model

In [ ]:
# Splitting data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.05, random_state=2)

# Training the metric learning model using KNN
# -- I will substitute KNN -- 
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)

In [ ]:
# Evaluating the model
accuracy = knn.score(x_test, y_test)
print("Score:", accuracy)

## Including new person

In [43]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MTCNN model for facial recognition
mtcnn = MTCNN(image_size=112, margin=0, min_face_size=20, thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, device=device)

# FaceNet model (InceptionResnetV1)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
# Function to extract facial embeddings using FaceNet
def extract_face_embeddings(img_path):
  img = Image.open(img_path).convert('RGB')
  img = img.resize((112, 112))
  img_cropped = mtcnn(img)
  if img_cropped is not None:
    with torch.no_grad():
      embedding = model(img_cropped.unsqueeze(0).to(device))
    return embedding.squeeze().cpu().numpy()
  return None

In [ ]:
# Including new person
emb = extract_face_embeddings('../data/newperson.jpg')
display(Image.open('../data/newperson.jpg'))
emb = emb.reshape(1, -1)
embeddings = np.vstack([embeddings, emb])
# Decoding the labels
labels = label_encoder.inverse_transform(labels)
labels = np.append(labels, 'New Person')
print(labels.shape)
print(f'{labels[-1]} was added!')
labels = label_encoder.fit_transform(labels)

In [ ]:
# Function to recognize the person given a new image
def recognize_person(img_path):
  img = Image.open(img_path).convert('RGB')
  img = img.resize((112, 112))
  img_cropped = mtcnn(img)
  if img_cropped is not None:
    with torch.no_grad():
      embedding = model(img_cropped.unsqueeze(0).to(device)).squeeze().cpu().numpy()
    predicted_label = knn.predict([embedding])[0]
    predicted_person = label_encoder.inverse_transform([predicted_label])[0]
    return predicted_person
  return None

In [ ]:
# Testing inference on a new image
new_image_path = '../data/newpersontest.jpg'
display(Image.open(new_image_path))
predicted_person = recognize_person(new_image_path)
print("Predicted person:", predicted_person)